# Configuration prioritization using QAOA

We want to extend our findings from the qaoa_mincost_sat notebook by adding functionality for creating an ordered list of configurations sorted by ascending configuration cost.
We will be using the same mincost SAT instance as before as an example.


The cost for each valid configuration is as follows:

| Feature Vector | Config. Cost | Feature Vector | Config. Cost |
|----------------|--------------|----------------|--------------|
| 01 01 01       | 80           | 01 10 00       | 45           |
| 10 01 01       | 90           | 10 10 00       | 55           |
| 11 01 01       | 110          | 11 10 00       | 75           |
| 01 10 01       | 55           | 01 01 11       | 90           |
| 10 10 01       | 65           | 10 01 11       | 100          |
| 11 10 01       | 95           | 11 01 11       | 120          |
| 01 01 00       | 70           | 01 10 11       | 65           |
| 10 01 00       | 80           | 10 10 11       | 75           |
| 11 01 00       | 100          | 11 10 11       | 95           |

In [ ]:
# Notebook Setup
from IPython.core import page
page.page = print

# Imports used for examples
import qaoa_config_prioritization as prio
from pprint import pprint

In [ ]:
# cost and sat individually
from qubovert import boolean_var

# define binary vars
x1, x2, x3, x4, x5, x6 = boolean_var('x1'), boolean_var('x2'), boolean_var('x3'), boolean_var('x4'), boolean_var('x5'), boolean_var('x6')

# SAT Penalty
alpha_sat = 1000 # 1e6

# SAT QUBO
sat_model = alpha_sat * (2 - x1 - x2 - x3 - x4 + x5 + x1 * x2 + 2 * x3 * x4 - x5 * x6)

# Equal probability sat model
# sat_model = alpha_sat * ( 0.6 * (1 - x1 - x2 + x1 * x2) + (x3 * x4) + (1 - x3 - x4 + x3 * x4) + 0.6 * (x5 - x5 * x6) )

# Cost QUBO
cost_model = 30*x1 + 20*x2 + 25*x3 + 50*x4 + 30*x5 + 10*x6

# Combine models
combined_model = sat_model +  cost_model
combined_hamiltonian = combined_model.to_quso()

In [ ]:
# Config cost for each valid config
config_cost = {"101010": 80, "101001":  90, "101011": 110,
               "100110": 55, "100101":  65, "100111":  95,
               "001010": 70, "001001":  80, "001011": 100,
               "000110": 45, "000101":  55, "000111":  75,
               "111010": 90, "111001": 100, "111011": 120,
               "110110": 65, "110101":  75, "110111":  95}

# Adjusting the Hamiltonian

In order to create an ordered list of configurations with ascending cost, we need to run the QAOA algorithm multiple times and adjust the hamiltonian for each run to exclude the config that was previously measured with the highest probability.
We do this by using Variational Quantum Deflation as described by [Higgott et al.](https://arxiv.org/abs/1805.08138)

## Deflating configurations
In order to determine the $k$-th best configuration, we need to exclude every other configuration in our hamiltonian.
To do this we can use the following formular:

$H_k := H + \sum_{i = 0}^{k - 1} \beta_i | i \rangle \langle i |$

where $i$ is the previously found $i$-th configuration of our optimization problem.

To guarantee a minimum hamiltonian energy at $E_k$ for the $k$-th configuration, we need to choose $\beta_i > E_k - E_i$.
We achieve this by running the algorithm multiple times and adjusting (i.e. doubling) $\beta_i$ starting from a predetermined value until we find a $\beta_i$ which is big enough.

In [ ]:
%psource prio.deflate_config

In [ ]:
prioritized_list = prio.config_prioritization(combined_hamiltonian, 3, 2600)

for item in prioritized_list:
    # Check config validity
    if config_cost.keys().__contains__(item):
        print("Valid config " + str(item) + " with cost: " + str(config_cost[item]))
    else:
        print("Invalid config")